In [3]:
import os
import pandas as pd
import json
import re

In [4]:
# Forms a logger data frame
def form_logger_df(power_data_logger_file: os.path):
    logger_df = pd.read_csv(power_data_logger_file, sep=r'\s+')
    logger_df = logger_df[['timestamp', 'voltage_V', 'current_A', 'temp_C_ema']]

    # Ensure proper data types
    logger_df['timestamp'] = logger_df['timestamp'].astype(float)
    logger_df['voltage_V'] = logger_df['voltage_V'].astype(float)
    logger_df['current_A'] = logger_df['current_A'].astype(float)
    logger_df['temp_C_ema'] = logger_df['temp_C_ema'].astype(float)

    return logger_df

In [5]:
def form_command_df(command_feedback_file: os.path):
    # List to store each command as a dictionary
    command_list = []

    # Regular expression to match the pattern of each image result in the output
    output_pattern = re.compile(
        r"Image: (?P<image_file>[^,]+), "
        r"Resolution: (?P<resolution_width>\d+)x(?P<resolution_height>\d+), "
        r"Framework: (?P<framework>[A-Z]+), "
        r"Set FPS: (?P<fps>[\d.]+), "
        r"Inference time: (?P<inference_time>[\d.]+) seconds, "
        r"Predicted class: (?P<predicted_class>\d+)"
    )
    
    # Regular expression to extract the model name from the command
    model_pattern = re.compile(r'run_model\.py\s+([^\s]+)')

    # Open the file and read line by line
    with open(command_feedback_file, 'r') as file:
        for line in file:
            # Parse each line as a JSON object
            command_data = json.loads(line.strip())
            
            # Extract the output field and find all matches for the image data
            output_data = command_data.get('output', '')
            matches = output_pattern.findall(output_data)
            
            # Extract the model name from the command
            model_match = model_pattern.search(command_data['command'])
            model_name = model_match.group(1) if model_match else None
            
            # For each match, create a new dictionary entry with the parsed fields
            for match in matches:
                # Calculate the updated end_time based on the formula
                fps = float(match[4])
                inference_time = float(match[5])
                start_time = float(command_data["start_time"])
                # Compute max(1/fps, inference_time)
                time_delta = max(1/fps, inference_time)
                # Calculate new end_time
                new_end_time = start_time + time_delta

                image_data = {
                    "client_id": command_data["client_id"],
                    "command": command_data["command"],
                    "model": model_name,  # Add the model name to the dictionary
                    "start_time": float(command_data["start_time"]),
                    "new_end_time": round(new_end_time, 3),  # Rounded to 3 decimal places (10ms precision)
                    "error": command_data["error"],
                    "image_file": match[0],
                    "resolution_width": int(match[1]),
                    "resolution_height": int(match[2]),
                    "framework": match[3],
                    "fps": fps,
                    "inference_time": inference_time,
                    "predicted_class": int(match[6])
                }
                # Append the image data to the command list
                command_list.append(image_data)
    
    # Convert the list of image data into a pandas DataFrame
    command_df = pd.DataFrame(command_list)

    # Optionally format the float display in the DataFrame with three decimal places
    pd.options.display.float_format = '{:.3f}'.format

    return command_df

In [3]:
logger_df_pt = form_logger_df('data_logger_pt.txt')
logger_df_hailo = form_logger_df('data_logger_hailo.txt')

In [5]:
command_df_pt = form_command_df("command_feedback_pt.txt")
command_df_hailo = form_command_df("command_feedback_hailo.txt")

In [6]:
# Function to aggregate power data for each command
def merge_and_aggregate(command_df, logger_df):
    # Create lists to store aggregated values
    voltage_agg = []
    current_agg = []
    temp_agg = []

    # Iterate through each row in the command DataFrame
    for idx, row in command_df.iterrows():
        # Select rows from the logger DataFrame where timestamp is between start_time and new_end_time
        mask = (logger_df['timestamp'] >= row['start_time']) & (logger_df['timestamp'] <= row['new_end_time'])
        subset = logger_df[mask]

        # If there are matching rows, aggregate their values (mean in this case)
        if not subset.empty:
            voltage_agg.append(subset['voltage_V'].mean())
            current_agg.append(subset['current_A'].mean())
            temp_agg.append(subset['temp_C_ema'].mean())
        else:
            # If no data is found in the logger for this time window, append NaN or a default value
            voltage_agg.append(None)
            current_agg.append(None)
            temp_agg.append(None)

    # Add the aggregated columns to the command_df
    command_df['avg_voltage_V'] = voltage_agg
    command_df['avg_current_A'] = current_agg
    command_df['avg_temp_C'] = temp_agg

    # Add power column
    command_df['power_W'] = command_df['avg_voltage_V'] * command_df['avg_current_A']

    # Remove not needed columns
    command_df = command_df.drop(columns=['client_id', 'command', 'error'])
    
    return command_df

In [7]:
merged_df_pt = merge_and_aggregate(command_df_pt, logger_df_pt)
merged_df_hailo = merge_and_aggregate(command_df_hailo, logger_df_hailo)

In [11]:
merged_df = pd.concat([merged_df_pt, merged_df_hailo], axis=0)
# Export the merged dataset to CSV
merged_df.to_csv('merged_dataset.csv', index=False)

In [9]:
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
pytorch_df = merged_df[merged_df['framework'] == 'PYTORCH']
hailo_df = merged_df[merged_df['framework'] == 'HAILO']

In [26]:
def aggregate_power_by_fps(data):
    # Group by fps and model
    grouped = data.groupby(['fps', 'model']).agg(
        mean_power=('power_W', 'mean')
    ).reset_index()
    return grouped

# Aggregate power consumption for PyTorch
pytorch_agg = aggregate_power_by_fps(pytorch_df)

# Aggregate power consumption for Hailo
hailo_agg = aggregate_power_by_fps(hailo_df)

In [27]:
# Function to plot and save the graphs as PNG
def plot_and_save_power_vs_fps(agg_data, framework_name, fps_range, plot_title, file_name):
    plt.figure(figsize=(10, 6))

    # Filter the data for the given FPS range
    filtered_data = agg_data[(agg_data['fps'] >= fps_range[0]) & (agg_data['fps'] <= fps_range[1])]

    # Get the list of unique models
    models = filtered_data['model'].unique()

    # Plot each model
    for model in models:
        model_data = filtered_data[filtered_data['model'] == model]
        plt.plot(model_data['fps'], model_data['mean_power'], marker='o', label=model)

    plt.xlabel('FPS (Frames per Second)')
    plt.ylabel('Power Consumption (W)')
    plt.title(plot_title)
    plt.legend(title='Model')
    plt.grid(True)

    # Save the plot as a PNG file
    plt.savefig(file_name, format='png', dpi=300)  # 300 dpi for high resolution
    plt.close()  # Close the figure after saving

# Define FPS ranges
fps_range_1 = (pytorch_agg['fps'].min(), 1)  # From min FPS to 1
fps_range_2 = (1, 10)  # From 1 to 10

# Save PyTorch plots to PNG
plot_and_save_power_vs_fps(pytorch_agg, 'PyTorch', fps_range_1, 'Power Consumption vs FPS (min to 1) for PyTorch', 'pytorch_min_to_1.png')
plot_and_save_power_vs_fps(pytorch_agg, 'PyTorch', fps_range_2, 'Power Consumption vs FPS (1 to 10) for PyTorch', 'pytorch_1_to_10.png')

# Save Hailo plots to PNG
plot_and_save_power_vs_fps(hailo_agg, 'Hailo', fps_range_1, 'Power Consumption vs FPS (min to 1) for Hailo', 'hailo_min_to_1.png')
plot_and_save_power_vs_fps(hailo_agg, 'Hailo', fps_range_2, 'Power Consumption vs FPS (1 to 10) for Hailo', 'hailo_1_to_10.png')


In [28]:
# REDO for new Pytorch Data:

new_pt_df = merge_and_aggregate(form_command_df('command_feedback_pt.txt'), form_logger_df('data_logger_pt.txt'))
new_pt_df = new_pt_df[new_pt_df['framework'] == 'PYTORCH']
new_pt_df.head()
pytorch_df = new_pt_df
pytorch_agg = aggregate_power_by_fps(pytorch_df)

# # Plot for PyTorch, FPS range: min to 1
# plot_power_vs_fps(pytorch_agg, 'PyTorch', fps_range_1, 'Power Consumption vs FPS (min to 1) for PyTorch')

# # Plot for PyTorch, FPS range: 1 to 10
# plot_power_vs_fps(pytorch_agg, 'PyTorch', fps_range_2, 'Power Consumption vs FPS (1 to 10) for PyTorch')

# # Plot for Hailo, FPS range: min to 1
# plot_power_vs_fps(hailo_agg, 'Hailo', fps_range_1, 'Power Consumption vs FPS (min to 1) for Hailo')

# # Plot for Hailo, FPS range: 1 to 10
# plot_power_vs_fps(hailo_agg, 'Hailo', fps_range_2, 'Power Consumption vs FPS (1 to 10) for Hailo')

In [29]:
# Redo for MOBILENETV3

pt_mbn_df = merge_and_aggregate(form_command_df('command_feedback_pt_mbn.txt'), form_logger_df('data_logger_pt_mbn.txt'))
pt_mbn_df = pt_mbn_df[pt_mbn_df['framework'] == 'PYTORCH']

# Replace the MOBILENETV3 data
pytorch_df = pytorch_df.drop(pytorch_df[pytorch_df['model'] == 'MOBILENETV3'].index)
pytorch_df = pd.concat([pytorch_df, pt_mbn_df], ignore_index=True)
pytorch_agg = aggregate_power_by_fps(pytorch_df)

FileNotFoundError: [Errno 2] No such file or directory: 'command_feedback_pt_mbn.txt'

In [ ]:
# Save PyTorch plots to PNG
plot_and_save_power_vs_fps(pytorch_agg, 'PyTorch', fps_range_1, 'Power Consumption vs FPS (0.1 to 1) for PyTorch', 'pytorch_min_to_1.png')
plot_and_save_power_vs_fps(pytorch_agg, 'PyTorch', fps_range_2, 'Power Consumption vs FPS (1 to 10) for PyTorch', 'pytorch_1_to_10.png')

# Save Hailo plots to PNG
plot_and_save_power_vs_fps(hailo_agg, 'Hailo', fps_range_1, 'Power Consumption vs FPS (0.1 to 1) for Hailo', 'hailo_min_to_1.png')
plot_and_save_power_vs_fps(hailo_agg, 'Hailo', fps_range_2, 'Power Consumption vs FPS (1 to 10) for Hailo', 'hailo_1_to_10.png')


In [ ]:
final_df = pd.concat([hailo_df, pytorch_df], ignore_index=True)
final_df = final_df.sort_values(by=['model', 'framework', 'start_time']).reset_index(drop=True)
final_df.to_csv('final_df.csv')
